In [1]:
import re

In [2]:
# Truyện ngắn: Cho tôi xin một vé đi tuổi thơ - Tác giả Nguyễn Nhật Ánh
!curl https://raw.githubusercontent.com/tvthanh0401/Text-Generation/main/dulieu.txt --output dulieu.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  173k  100  173k    0     0  6660k      0 --:--:-- --:--:-- --:--:-- 6660k


In [3]:
with open('dulieu.txt', 'r', encoding='utf-8') as f:
  text = f.read()

In [4]:
new_text = re.sub('[:;()\*\-\"!]', ' ', text.lower())

In [5]:
tmp = new_text.split()
words = []
for w in tmp:
  if w[-1] == '.' or w[-1] == ',':
    words.append(w[:-1])
    words.append(w[-1])
  else:
    words.append(w)
  

In [6]:
words_set = list(set(words))

In [7]:
words_set.sort()

In [8]:
words_set[:10]

['', ',', '.', '..', '...............', '10', '15', '2', '2002', '22']

In [9]:
words[:10]

['một', 'ngày', ',', 'tôi', 'chợt', 'nhận', 'thấy', 'cuộc', 'sống', 'thật']

In [10]:
lookup = {word : index for index, word in enumerate(words_set)}
index_word_lookup = {index: word for index, word in enumerate(words_set)}

In [11]:
len(lookup)

2675

In [12]:
len(words)

32129

In [13]:
training = [lookup[word] for word in words]

In [14]:
training[:10]

[1216, 1278, 1, 2134, 305, 1402, 1923, 331, 1807, 1932]

In [15]:
index_word_lookup[934]

'lang'

In [16]:
seq_len = 20
dataX = []
dataY = []
for i in range(0, len(training) - seq_len):
  seq_in = words[i: i + seq_len]
  seq_out = words[i + seq_len]
  dataX.append(training[i: i + seq_len])
  dataY.append(training[i + seq_len])
print(len(dataX), len(dataY))

32109 32109


In [17]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [18]:
X = np.reshape(dataX, (len(dataX), seq_len, 1))
# normalize
X = X / float(len(lookup))
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [19]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [20]:
filepath="model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=200, batch_size=128, callbacks=callbacks_list)

Epoch 1/200
251/251 [==============================] - 19s 9ms/step - loss: 6.7385

Epoch 00001: loss improved from inf to 6.52871, saving model to model.hdf5
Epoch 2/200
251/251 [==============================] - 2s 9ms/step - loss: 6.3721

Epoch 00002: loss improved from 6.52871 to 6.37106, saving model to model.hdf5
Epoch 3/200
251/251 [==============================] - 2s 9ms/step - loss: 6.3551

Epoch 00003: loss improved from 6.37106 to 6.36352, saving model to model.hdf5
Epoch 4/200
251/251 [==============================] - 2s 9ms/step - loss: 6.3554

Epoch 00004: loss improved from 6.36352 to 6.36019, saving model to model.hdf5
Epoch 5/200
251/251 [==============================] - 2s 9ms/step - loss: 6.3419

Epoch 00005: loss improved from 6.36019 to 6.35687, saving model to model.hdf5
Epoch 6/200
251/251 [==============================] - 2s 9ms/step - loss: 6.3397

Epoch 00006: loss improved from 6.35687 to 6.35567, saving model to model.hdf5
Epoch 7/200
251/251 [==========

In [23]:
filename = 'model.hdf5'
model.load_weights(filename)
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ' '.join([index_word_lookup[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(len(lookup))
    prediction = model.predict(x, verbose=0)
    #print(prediction[:100])
    index = np.argmax(prediction)
    #print(index)
    result = index_word_lookup[index]
    #seq_in = [index_word_lookup[value] for value in pattern]
    print(result)
    pattern.append(index)
    pattern = pattern[1:]
print ("\nDone.")

Seed:
" thế , người israel và người palestine nã pháo vào nhau chắc chắn không bắt nguồn từ việc bọn tôi "
thay
chén
bằng
thau
,
nhưng
dưới
mắt
các
ông
bố
bà
mẹ
là
việc
hôm
cò
,
ba
nghĩa
là
như
,
tôi
tôi
gặp
nó
hải
cò
nó
gì
.
cho
tôi
từng
,
em
bỗng
cần
,
chỉ
tủn
nhà
tôi
con
nhiên
gửi
cò
tôi
,
con
tủn
là
nó
chiếc
tôi
chiếc
mươi
.
tôi
khi
là
có
.
tôi
tôi
.
vẫn
tôi
mắt
,
giờ
đó
,
,
con
tí
sún
con
tí
nhà
,
cho
giờ
,
tôi
cái
là
phải
nào
,
tôi
như
,
nguyên
ra
trong
,
con
biết
tôi
thở
là
do
tôi
tôi
.
con
,
thế
không
hả
con?
.
tôi
một
tôi
bình
không
con
để
ba
tôi
.
là
chắc
là
ba
.
cái
tôi
đần
,
con
tủn
,
hải
cò
.
tôi
là
là
phải
con
tủn
,
,
có
.
chạy
xếp
gọn
gàng
.
tôi
.
kho
chữ
con
con
thể
,
tôi
chỉ
chân
là
cái
mẹ
.
–
là
đến
một
chút
đồng
sún
,
con
nhấn
chồng
con
bỗng
,
hải
là
một
,
tôi
công
động
một
khỏi
do
lấy
,
tôi
là
tôi
và
,
cũng
tủn
sún
,
con
gái
–
tôi
và
con
xinh
,
.
tôi
,
cũng
đó
,
tôi
,
,
lần
.
và
không
giống
cảnh
từ
,
,
tôi
là
con
tí
,
bạn
tuổi
bạn
con
.
.
buồn
cũng
cam
chứ
sức
ra
gi